### Code in v10 works for getting HOF eligible players for training. But need to incorporate current MLB players.

In [1]:
import pandas as pd
import numpy as np
import pickle

/Applications/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [16]:
def calculate_stat_ratio(row, stat, denom):
    '''
    Description: Calculate the ratio of a player's stats in a particular category to the mean, median, or min of
    that stat for a Hall of Famer at that same position
    '''
    return ((row[stat] / row['year']) * (row['year'] + row['yrs_remain'])) / row[denom]

def create_eras_cols(df):
    # Dead Ball Era 1 (1900-1919)
    df['DBE1'] = 0
    df.ix[(df['yearID'] >= 1900) & (df['yearID'] <= 1919), 'DBE1'] = 1

    # Dead Ball Era 2 (1961-1968)
    df['DBE2'] = 0
    df.ix[(df['yearID'] >= 1961) & (df['yearID'] <= 1968), 'DBE2'] = 1

    # Steroid era (1988-2003)
    df['SE'] = 0
    df.ix[(df['yearID'] >= 1988) & (df['yearID'] <= 2003), 'SE'] = 1
    return df

def combine_stints(df):
    df2 = df.copy()
    if 'inducted' in df2.columns:
        df2 = df2.drop('inducted', axis=1)
    if 'stint' in df2.columns:
        df2 = df2.drop('stint', axis=1)
    return df2.groupby(['playerID', 'yearID']).sum().reset_index().sort_values(by=['playerID', 'yearID'])


def create_stat_ratio_cols(df, stats_of_interest, denominator='mean'):
    '''
    Description: Calculate the desired stats ratios and add them as new column to df
    '''
    if denominator == 'mean':
        denom_stats = [stat + '_mean' for stat in stats_of_interest]
    elif denominator == 'median':
        denom_stats = [stat + '_med' for stat in stats_of_interest]
    elif denominator == 'min':
        denom_stats = [stat + '_min' for stat in stats_of_interest]
        
    stats_ratio = [stat + '_ratio' for stat in stats_of_interest]

    for stat, denom, stat_ratio in zip(stats_of_interest, denom_stats, stats_ratio):
        df[stat_ratio] = df.apply(calculate_stat_ratio, axis=1, args=(stat, denom))
    return df

def create_yr_col(df):
    # Create 'year' variable indicating the number of years players have played in the MLB.
    player_startyr_dict = pd.DataFrame(df.groupby('playerID').min()['yearID']).to_dict()['yearID']
    df['year'] = df.apply(subtract_start_yr, axis=1, args=(player_startyr_dict,))
    return df

# def create_remain_yrs_col(df):
#     # Create 'yrs_remain' variable that estimates the remaining number of years for that player.
#     # Based on the median number of years that eligible players at that position played
#     player_totalyrs_dict = pd.DataFrame(df.groupby('playerID').max()['year']).to_dict()['year']
#     df['yrs_remain'] = df.apply(get_remaining_yrs, axis=1, args=(player_totalyrs_dict,))
#     return df

def fill_na(df, stats_of_interest):
    stats_filled = []
    for stat in stats_of_interest:
        if df[stat].isnull().sum() > 0:
            stat_filled = stat + '_filled'
            df[stat_filled] = 0
            df.ix[df[stat].isnull(), stat_filled] = 1
            df.ix[df[stat].isnull(), stat] = 0
            stats_filled.append(stat_filled)
    return df, stats_filled

def get_birth_year(filename):
    master = pd.read_csv(filename)
    return pd.DataFrame(master.groupby('playerID').sum()['birthYear']).reset_index()

def get_cumulative_stats(df):
    # Calculate cumulative stats over the years for each player.
    stats = ['G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', \
             'IBB', 'HBP', 'SH', 'SF', 'GIDP'] 
    return df.groupby('playerID')[stats].cumsum()[stats]

# def get_cumulative_stats(df):
#     # Calculate cumulative stats over the years for each player.
#     stats = ['G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', \
#              'IBB', 'HBP', 'SH', 'SF', 'GIDP']
#     return df.groupby('playerID')[stats].cumsum()[stats]


def get_hof_labels(filename):
    '''
    INPUT: 1 file
    OUTPUT: Pandas df

    Given Hall of Fame (HOF) data file, create HOF labels for all eligible players (inducted and not inducted)
    Returns: dataframe of all eligible HOF players with labels indicating if they were inducted or not.
    '''

    # Load HallofFame.csv file containing players who were/are eligible for election to HOF.
    hof = pd.read_csv(filename)

    # Select those who were inducted into HOF
    hof_players = hof[(hof['inducted'] == 'Y') & (hof['category'] == 'Player')][['playerID', 'inducted']]
    hof_players['inducted'] = hof_players['inducted'].map({'Y' : 1})

#    hof_player_indices = set(hof_players.index)
    hof_playerID = set(hof_players['playerID'])

    # Select all eligible players for the HOF (i.e., those who were on the ballot)
    elig = hof[(hof['category'] == 'Player')]

#    elig_indices = set(elig.index)
    elig_playerID = set(elig['playerID'])

    # Select players who were on the ballot but were not inducted into HOF
    nonhof_playerID = elig_playerID - hof_playerID
    nonhof_playerID = list(nonhof_playerID)
    nonhof_players = pd.DataFrame(nonhof_playerID, columns=['playerID'])
    nonhof_players['inducted'] = 0

    # Merge hof_players and nonhof_players
    return pd.concat([hof_players, nonhof_players])

def get_hofer_stats(df, stats_of_interest, calculate='mean'):
    '''
    Description: Determine mean, median, or min of each stat for HOF players at each position
    '''
#     stats = ['G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', \
#              'IBB', 'HBP', 'SH', 'SF', 'GIDP']

    positions = df['POS'].unique()
    position_stats_lst = []

    for position in positions:
        pos = pd.Series([position], index=['POS'])
        if calculate == 'mean':
            stats_labels = [stat + '_mean' for stat in stats_of_interest]
            stats_labels.append('POS')
            position_stats = df[(df['inducted'] == 1) & (df['POS'] == position)]\
            .groupby('playerID')[stats_of_interest].max().mean().round(1).append(pos)
        elif calculate == 'median':
            stats_labels = [stat + '_med' for stat in stats_of_interest]
            stats_labels.append('POS')
            position_stats = df[(df['inducted'] == 1) & (df['POS'] == position)]\
            .groupby('playerID')[stats_of_interest].max().median().round(1).append(pos)
        elif calculate == 'min':
            stats_labels = [stat + '_min' for stat in stats_of_interest]
            stats_labels.append('POS')
            position_stats = df[(df['inducted'] == 1) & (df['POS'] == position)]\
            .groupby('playerID')[stats_of_interest].max().min().round(1).append(pos)            
        
        position_stats_lst.append(position_stats)

    hofer_stats_df = pd.DataFrame(position_stats_lst)

    hofer_stats_df.columns = stats_labels
    return hofer_stats_df


def get_positions(filename):

    # Load Fielding.csv file
    fielding = pd.read_csv(filename)

    # Set position of each player to the one at which he played the most games
    fielding_grouped = fielding.groupby(['playerID', 'POS']).sum().reset_index()
    max_game_indices = np.array(fielding_grouped.groupby('playerID')['G'].idxmax())
    player_pos = fielding_grouped.iloc[max_game_indices][['playerID', 'POS']]

    # Set all outfield positions (LF, CF, OF) to OF.
    positions_dict = {'P': 'P', 'OF': 'OF', '1B': '1B', '2B': '2B', 'C': 'C', 'SS': 'SS', \
                    '3B': '3B', 'DH': 'DH', 'CF': 'OF', 'LF': 'OF'}
    player_pos['POS'] = player_pos['POS'].map(positions_dict)

    # # Write out player position dataframe
    # with open('player_pos.pkl', 'w') as f:
    #     pickle.dump(player_pos, f)
    #
    return player_pos


def get_remaining_yrs(row, retire_age='mean'):
    if retire_age == 'mean':      
        yrs_remain = row['retire_age_mean'] - row['age']
    elif retire_age == 'median':
        yrs_remain = row['retire_age_med'] - row['age']        
    elif retire_age == 'max':
        yrs_remain = row['retire_age_max'] - row['age']        
    if yrs_remain >= 0:
        return yrs_remain
    # In case player has played longer than mean career, set to 0.
    else:
        return 0

def get_retirement_age(df, calculate='mean'):
    '''
    Description: Get mean or median retirement age of all eligible players at the different positions 
    '''    
    positions = df['POS'].unique()    
    df_age = pd.DataFrame()
    
    for position in positions:
        pos = pd.Series([position], index=['POS'])
        if calculate == 'mean':
            retirement_age = round(df[df['POS'] == position].groupby('playerID')['age'].max().mean(), 1)
            col = 'retire_age_mean'         
        elif calculate == 'median':
            retirement_age = round(df[df['POS'] == position].groupby('playerID')['age'].max().median(), 1)
            col = 'retire_age_med'
        elif calculate == 'max':
            retirement_age = round(df[df['POS'] == position].groupby('playerID')['age'].max().max(), 1)
            col = 'retire_age_max'
        
        df_age = df_age.append({'POS': position, col: retirement_age}, ignore_index=True)
    return df_age


def subtract_start_yr(row, player_dict):
    '''
    Description: Subtract start year from each player's yearID 
    '''    
    name = row['playerID']
    return row['yearID'] - player_dict[name] + 1


# # -------------------------------------------------------------------

In [17]:

batting_file = '../BaseballHOF-repo/data/SeanLahmanBaseballDB/baseballdatabank-master/core/Batting.csv'
fielding_file = '../BaseballHOF-repo/data/SeanLahmanBaseballDB/baseballdatabank-master/core/Fielding.csv'
hall_of_fame_file = '../BaseballHOF-repo/data/SeanLahmanBaseballDB/baseballdatabank-master/core/HallOfFame.csv'
master_file = '../BaseballHOF-repo/data/SeanLahmanBaseballDB/baseballdatabank-master/core/Master.csv'

# 2. Get and join player positions to the df
player_pos = get_positions(fielding_file)
batting = pd.read_csv(batting_file)
all_players = batting.merge(player_pos, on='playerID')

# 3. Select only hitters (remove pitchers from df)
all_hitters = all_players[all_players['POS'] != 'P']

# 4. Some players played a stint for different teams in the same season.
# Combine the stats in those cases into one row.
combined_stints = combine_stints(all_hitters)

# Select which stats to include in model
stats_of_interest = ['R', 'H', 'HR', 'RBI', 'SB', 'BB']

# 5. Fill in null values that are present in the stats of interest
filled_na_df, filled_stats = fill_na(combined_stints, stats_of_interest)

# 6a. Calculate cumulative stats over the years for each player.
cumulative_stats = get_cumulative_stats(filled_na_df)
# 6b. Combine cumulative stats with non-stats columns
cols_to_add = list(set(filled_na_df.columns) - set(cumulative_stats.columns))
all_hitters_cumstats = cumulative_stats.join(filled_na_df[cols_to_add])[filled_na_df.columns]

# 7. Get and join birth year of each player to df and create 'age' column
birth_year = get_birth_year(master_file)
all_hitters_cumstats = all_hitters_cumstats.merge(birth_year, on='playerID')
all_hitters_cumstats['age'] = all_hitters_cumstats['yearID'] - all_hitters_cumstats['birthYear']

# 11. Create 'year' variable indicating the number of years players have played in MLB.
all_hitters_cumstats = create_yr_col(all_hitters_cumstats)

# 8. Re-join positions to df
all_hitters_cumstats = all_hitters_cumstats.merge(player_pos, on='playerID')

# 13. Add variables corresponding to different baseball eras
all_hitters_cumstats = create_eras_cols(all_hitters_cumstats)

# --------------------------------------------------------------------------------------------

## Create dataframe of HOF eligible hitters.

# Select only players from batting table who were/are eligible for HOF.
elig_labels = get_hof_labels(hall_of_fame_file)

# Drop a few players from the list (brownwi02, irvinmo01, tennefr01). Willard Brown (brownwi02) and Monte Irvin
# (irvinmo01) were both inducted into the HOF, but they played the majority of their careers in the Negro League
# for which stats are not available. Fred Tenney was nominated but only one years worth of stat is available.

omit = ['brownwi02', 'irvinmo01', 'tennefr01']
elig_labels = elig_labels[-elig_labels['playerID'].isin(omit)]

# Also, Jacque Jones' playerID is jonesja05 in the HallofFame file. This corresponds to only one year's worth of stats.
# Majority of his career stats are associated with playerID jonesja04. Replace with this playerID instead.

elig_labels.loc[elig_labels['playerID'] == 'jonesja05', 'playerID'] = 'jonesja04'


# 1. Merge hof labels with all_hitters_cumstats df
elig_hitters_cumstats = all_hitters_cumstats.merge(elig_labels, on='playerID')


# 9. Get mean, median, or min stats of different positions for HOF hitters and merge to dfs
hof_hitters_stats = get_hofer_stats(elig_hitters_cumstats, stats_of_interest, 'mean')

all_hitters_cumstats = all_hitters_cumstats.merge(hof_hitters_stats, on='POS')
elig_hitters_cumstats = elig_hitters_cumstats.merge(hof_hitters_stats, on='POS')

# 10. Get mean or median retirement age of different positions for all eligible HOF hitters and merge to dfs
retirement_age_elig = get_retirement_age(elig_hitters_cumstats, 'mean')

all_hitters_cumstats = all_hitters_cumstats.merge(retirement_age_elig, on='POS')
elig_hitters_cumstats = elig_hitters_cumstats.merge(retirement_age_elig, on='POS')


# --------------------------------------------------------------------------------------------

# Steps 12 and 14 done for both df_all and df_hof_elig

# 12. Create 'yrs_remain' column that estimates the remaining number of years in the career of that player
# based on the mean retirement age of eligible players at that position
all_hitters_cumstats['yrs_remain'] = all_hitters_cumstats.apply(get_remaining_yrs, axis=1, \
                                                                  args=('mean',))
elig_hitters_cumstats['yrs_remain'] = elig_hitters_cumstats.apply(get_remaining_yrs, axis=1, \
                                                                  args=('mean',))

# 14. Calculate the ratio of a player's cumulative total for a particular stat to the mean, median, or min of that
# stat for players at that position who are in the HOF
all_hitters_ratios = create_stat_ratio_cols(all_hitters_cumstats, stats_of_interest, 'mean')
elig_hitters_ratios = create_stat_ratio_cols(elig_hitters_cumstats, stats_of_interest, 'mean')



In [19]:
elig_hitters_ratios.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11617 entries, 0 to 11616
Data columns (total 43 columns):
playerID           11617 non-null object
yearID             11617 non-null int64
G                  11617 non-null int64
AB                 11617 non-null float64
R                  11617 non-null float64
H                  11617 non-null float64
2B                 11617 non-null float64
3B                 11617 non-null float64
HR                 11617 non-null float64
RBI                11617 non-null float64
SB                 11617 non-null float64
CS                 8637 non-null float64
BB                 11617 non-null float64
SO                 10741 non-null float64
IBB                6788 non-null float64
HBP                11386 non-null float64
SH                 11086 non-null float64
SF                 6848 non-null float64
GIDP               8143 non-null float64
RBI_filled         11617 non-null int64
SB_filled          11617 non-null int64
birthYear          116

In [18]:
elig_hitters_ratios.head(30)

,playerID,yearID,G,AB,R,H,2B,3B,HR,RBI,...,SB_mean,BB_mean,retire_age_mean,yrs_remain,R_ratio,H_ratio,HR_ratio,RBI_ratio,SB_ratio,BB_ratio
0,aaronha01,1954,122,468.0,58.0,131.0,27.0,6.0,13.0,69.0,...,268.1,959.9,37.3,17.3,0.723025,0.932838,0.950459,0.968180,0.136516,0.533806
1,aaronha01,1955,275,1070.0,163.0,320.0,64.0,15.0,40.0,175.0,...,268.1,959.9,37.3,16.3,1.015974,1.139344,1.462245,1.227764,0.170645,0.733983
2,aaronha01,1956,428,1679.0,269.0,520.0,98.0,29.0,66.0,267.0,...,268.1,959.9,37.3,15.3,1.117779,1.234289,1.608470,1.248812,0.159269,0.724450
3,aaronha01,1957,579,2294.0,387.0,718.0,125.0,35.0,110.0,399.0,...,268.1,959.9,37.3,14.3,1.206080,1.278201,2.010587,1.399651,0.136516,0.815007
4,aaronha01,1958,732,2895.0,496.0,914.0,159.0,39.0,140.0,494.0,...,268.1,959.9,37.3,13.3,1.236621,1.301700,2.047143,1.386321,0.163819,0.876966
5,aaronha01,1959,886,3524.0,612.0,1137.0,205.0,46.0,179.0,617.0,...,268.1,959.9,37.3,12.3,1.271526,1.349410,2.181183,1.442915,0.227527,0.892853
6,aaronha01,1960,1039,4114.0,714.0,1309.0,225.0,57.0,219.0,743.0,...,268.1,959.9,37.3,11.3,1.271526,1.331608,2.287369,1.489353,0.351042,0.928713
7,aaronha01,1961,1194,4717.0,829.0,1506.0,264.0,67.0,253.0,863.0,...,268.1,959.9,37.3,10.3,1.291783,1.340509,2.312175,1.513658,0.486339,0.946075
8,aaronha01,1962,1350,5309.0,956.0,1697.0,292.0,73.0,298.0,991.0,...,268.1,959.9,37.3,9.3,1.324160,1.342685,2.420828,1.545034,0.546065,0.980762
9,aaronha01,1963,1511,5940.0,1077.0,1898.0,321.0,77.0,342.0,1121.0,...,268.1,959.9,37.3,8.3,1.342582,1.351547,2.500439,1.572941,0.703059,1.031389


In [94]:
get_retirement_age_test(elig_hitters_cumstats, 'median')

,POS,retire_age_med
0,OF,37.0
1,3B,36.0
2,2B,36.0
3,1B,37.0
4,SS,37.0
5,C,37.0
6,DH,41.0


In [5]:
fielding = pd.read_csv('../BaseballHOF-repo/data/SeanLahmanBaseballDB/baseballdatabank-master/core/Fielding.csv')

In [6]:
batting = pd.read_csv('../BaseballHOF-repo/data/SeanLahmanBaseballDB/baseballdatabank-master/core/Batting.csv')

In [7]:
len(fielding['playerID'].unique())

18465

In [10]:
fielding['POS'].unique()

array(['SS', '2B', 'OF', 'C', '1B', '3B', 'P', 'LF', 'RF', 'CF', 'DH'], dtype=object)

In [8]:
len(batting['playerID'].unique())

18659

In [53]:
set(batting['playerID'].unique()) - set(fielding['playerID'].unique())

{'aragoja01',
 'atkinle01',
 'banisje01',
 'barbare01',
 'bassdo01',
 'batscbi01',
 'belnovi01',
 'biglepe01',
 'bluhmre01',
 'bowmael01',
 'brovijo01',
 'brownde01',
 'bubseha01',
 'burkach01',
 'burnscb01',
 'byrdji01',
 'campbji02',
 'casimca01',
 'cassija01',
 'clarkgl01',
 'cobbjo01',
 'connejo01',
 'corrijo01',
 'cortaje01',
 'coseyra01',
 'cotepe01',
 'creedco01',
 'daubeha01',
 'daughbo01',
 'daughdo01',
 'davisot01',
 'decasyu01',
 'delarje01',
 'diazel01',
 'dickepa01',
 'duffpa01',
 'dwyerdo01',
 'eastojo01',
 'echoljo01',
 'eversjo02',
 'falloch01',
 'falsepe01',
 'fautsjo01',
 'fetzewi01',
 'fialane01',
 'fishewi01',
 'fitzbch01',
 'fitzgra01',
 'fletcfr01',
 'foleyra01',
 'fostere01',
 'fritzla01',
 'gablega01',
 'gaedeed01',
 'gaglira01',
 'galviji01',
 'garboal01',
 'garrice01',
 'genovge01',
 'gentisa01',
 'gentrha01',
 'gleasro01',
 'godwity01',
 'goletst01',
 'gormahe01',
 'greenad01',
 'greenjo02',
 'hajduch01',
 'hamribe01',
 'hansedo01',
 'hardgpa01',
 'hargiga01'

In [22]:
pd.options.display.max_columns = 999

In [23]:
all_hitters_cumstats.head(30)

,playerID,yearID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,RBI_filled,SB_filled,birthYear,age,year,POS,DBE1,DBE2,SE,R_mean,H_mean,HR_mean,RBI_mean,SB_mean,BB_mean,retire_age_mean,yrs_remain,R_ratio,H_ratio,HR_ratio,RBI_ratio,SB_ratio,BB_ratio
0,aaronha01,1954,122,468.0,58.0,131.0,27.0,6.0,13.0,69.0,2.0,2.0,28.0,39.0,NaN,3.0,6.0,4.0,13.0,0,0,1934.0,20.0,1,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,17.3,0.723025,0.932838,0.950459,0.968180,0.136516,0.533806
1,aaronha01,1955,275,1070.0,163.0,320.0,64.0,15.0,40.0,175.0,5.0,3.0,77.0,100.0,5.0,6.0,13.0,8.0,33.0,0,0,1934.0,21.0,2,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,16.3,1.015974,1.139344,1.462245,1.227764,0.170645,0.733983
2,aaronha01,1956,428,1679.0,269.0,520.0,98.0,29.0,66.0,267.0,7.0,7.0,114.0,154.0,11.0,8.0,18.0,15.0,54.0,0,0,1934.0,22.0,3,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,15.3,1.117779,1.234289,1.608470,1.248812,0.159269,0.724450
3,aaronha01,1957,579,2294.0,387.0,718.0,125.0,35.0,110.0,399.0,8.0,8.0,171.0,212.0,26.0,8.0,18.0,18.0,67.0,0,0,1934.0,23.0,4,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,14.3,1.206080,1.278201,2.010587,1.399651,0.136516,0.815007
4,aaronha01,1958,732,2895.0,496.0,914.0,159.0,39.0,140.0,494.0,12.0,9.0,230.0,261.0,42.0,9.0,18.0,21.0,88.0,0,0,1934.0,24.0,5,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,13.3,1.236621,1.301700,2.047143,1.386321,0.163819,0.876966
5,aaronha01,1959,886,3524.0,612.0,1137.0,205.0,46.0,179.0,617.0,20.0,9.0,281.0,315.0,59.0,13.0,18.0,30.0,107.0,0,0,1934.0,25.0,6,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,12.3,1.271526,1.349410,2.181183,1.442915,0.227527,0.892853
6,aaronha01,1960,1039,4114.0,714.0,1309.0,225.0,57.0,219.0,743.0,36.0,16.0,341.0,378.0,72.0,15.0,18.0,42.0,115.0,0,0,1934.0,26.0,7,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,11.3,1.271526,1.331608,2.287369,1.489353,0.351042,0.928713
7,aaronha01,1961,1194,4717.0,829.0,1506.0,264.0,67.0,253.0,863.0,57.0,25.0,397.0,442.0,92.0,17.0,19.0,51.0,131.0,0,0,1934.0,27.0,8,OF,0,1,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,10.3,1.291783,1.340509,2.312175,1.513658,0.486339,0.946075
8,aaronha01,1962,1350,5309.0,956.0,1697.0,292.0,73.0,298.0,991.0,72.0,32.0,463.0,515.0,106.0,20.0,19.0,57.0,145.0,0,0,1934.0,28.0,9,OF,0,1,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,9.3,1.324160,1.342685,2.420828,1.545034,0.546065,0.980762
9,aaronha01,1963,1511,5940.0,1077.0,1898.0,321.0,77.0,342.0,1121.0,103.0,37.0,541.0,609.0,124.0,20.0,19.0,62.0,156.0,0,0,1934.0,29.0,10,OF,0,1,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,8.3,1.342582,1.351547,2.500439,1.572941,0.703059,1.031389


In [100]:

# --------------------------------------------------------------------------------------------

# 15. Filter all_hitters_ratios df for just active players and those still on the HOF ballot

# Get most current year of database
most_current_yr = all_hitters_ratios[['playerID', 'yearID']].groupby('playerID').max()['yearID'].max()

# Get the last year in which a player has played
all_hitters_ratios_last_yr = all_hitters_ratios[['playerID', 'yearID']].groupby('playerID').max().reset_index()

# Select only for hitters that are currently still in MLB
active_hitters = all_hitters_ratios_last_yr[all_hitters_ratios_last_yr['yearID'] == most_current_yr]['playerID'].values
active_hitters_ratios = all_hitters_ratios[all_hitters_ratios['playerID'].isin(active_players)]

with open('active_hitters.pkl', 'w') as f:
    pickle.dump(active_hitters_ratios['playerID'], f)


# Select the stat ratio columns as feature set on which to train model and write out.
# Feature set 1
features = [stat + '_ratio' for stat in stats_of_interest] + filled_stats
X1 = elig_hitters_ratios[features]
with open('eligible_hitters_X1.pkl', 'w') as f:
    pickle.dump(X1, f)

# Feature set 2
features2 = features = [stat + '_ratio' for stat in stats_of_interest] + ['year'] + filled_stats
X2 = elig_hitters_ratios[features2]
with open('eligible_hitters_X2.pkl', 'w') as f:
    pickle.dump(X2, f)

# Feature set 3
features3 = features = [stat + '_ratio' for stat in stats_of_interest] + ['year', 'yearID'] + filled_stats
X3 = elig_hitters_ratios[features3]
with open('eligible_hitters_X3.pkl', 'w') as f:
    pickle.dump(X3, f)

# Feature set 4
features4 = features = [stat + '_ratio' for stat in stats_of_interest] + ['year', 'yearID', 'DBE1',
                                                                         'DBE2', 'SE'] + filled_stats
X4 = elig_hitters_ratios[features4]
with open('eligible_hitters_X4.pkl', 'w') as f:
    pickle.dump(X4, f)

# Select 'inducted' column as target variable (1 = inducted into HOF, 0 = not inducted into HOF)
y = elig_hitters_ratios['inducted']
with open('eligible_hitters_y.pkl', 'w') as f:
    pickle.dump(y, f)


# Select the stat ratio columns as feature set for the active players
active1 = active_hitters_ratios[features]
with open('active_hitters_X1.pkl', 'w') as f:
    pickle.dump(active1, f)

active2 = active_hitters_ratios[features2]
with open('active_hitters_X2.pkl', 'w') as f:
    pickle.dump(active2, f)

active3 = active_hitters_ratios[features3]
with open('active_hitters_X3.pkl', 'w') as f:
    pickle.dump(active3, f)

active4 = active_hitters_ratios[features4]
with open('active_hitters_X_4.pkl', 'w') as f:
    pickle.dump(active4, f)


In [62]:
all_hitters_ratios[['playerID', 'yearID']].groupby('playerID').max()['yearID'].max()

2015

In [63]:
most_current_yr = all_hitters_ratios[['playerID', 'yearID']].groupby('playerID').max()['yearID'].max()

In [64]:
most_current_yr

2015

In [58]:
all_hitters_ratios.head(5)

,playerID,yearID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,RBI_filled,SB_filled,birthYear,age,year,POS,DBE1,DBE2,SE,R_mean,H_mean,HR_mean,RBI_mean,SB_mean,BB_mean,retire_age_mean,yrs_remain,R_ratio,H_ratio,HR_ratio,RBI_ratio,SB_ratio,BB_ratio
0,aaronha01,1954,122,468.0,58.0,131.0,27.0,6.0,13.0,69.0,2.0,2.0,28.0,39.0,NaN,3.0,6.0,4.0,13.0,0,0,1934.0,20.0,1,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,17.3,0.723025,0.932838,0.950459,0.968180,0.136516,0.533806
1,aaronha01,1955,275,1070.0,163.0,320.0,64.0,15.0,40.0,175.0,5.0,3.0,77.0,100.0,5.0,6.0,13.0,8.0,33.0,0,0,1934.0,21.0,2,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,16.3,1.015974,1.139344,1.462245,1.227764,0.170645,0.733983
2,aaronha01,1956,428,1679.0,269.0,520.0,98.0,29.0,66.0,267.0,7.0,7.0,114.0,154.0,11.0,8.0,18.0,15.0,54.0,0,0,1934.0,22.0,3,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,15.3,1.117779,1.234289,1.608470,1.248812,0.159269,0.724450
3,aaronha01,1957,579,2294.0,387.0,718.0,125.0,35.0,110.0,399.0,8.0,8.0,171.0,212.0,26.0,8.0,18.0,18.0,67.0,0,0,1934.0,23.0,4,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,14.3,1.206080,1.278201,2.010587,1.399651,0.136516,0.815007
4,aaronha01,1958,732,2895.0,496.0,914.0,159.0,39.0,140.0,494.0,12.0,9.0,230.0,261.0,42.0,9.0,18.0,21.0,88.0,0,0,1934.0,24.0,5,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,13.3,1.236621,1.301700,2.047143,1.386321,0.163819,0.876966


In [70]:
all_hitters_ratios_last_yr = all_hitters_ratios[['playerID', 'yearID']].groupby('playerID').max().reset_index()

In [71]:
all_hitters_ratios_last_yr

,playerID,yearID
0,aaronha01,1976
1,aaronto01,1971
2,abadan01,2006
3,abadijo01,1875
4,abbated01,1910
5,abbeych01,1897
6,abbotfr01,1905
7,abbotje01,2001
8,abbotku01,2001
9,abbotod01,1910


In [83]:
active_hitters = all_hitters_ratios_last_yr[all_hitters_ratios_last_yr['yearID'] == most_current_yr]['playerID'].values

In [85]:
active_hitters_ratios = all_hitters_ratios[all_hitters_ratios['playerID'].isin(active_players)]

In [90]:
len(active_hitters_ratios['playerID'].unique())

633

In [87]:
active_hitters_ratios[active_hitters_ratios['playerID'] == 'mccutan01']

,playerID,yearID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,RBI_filled,SB_filled,birthYear,age,year,POS,DBE1,DBE2,SE,R_mean,H_mean,HR_mean,RBI_mean,SB_mean,BB_mean,retire_age_mean,yrs_remain,R_ratio,H_ratio,HR_ratio,RBI_ratio,SB_ratio,BB_ratio
11685,mccutan01,2009,108,433.0,74.0,124.0,26.0,9.0,12.0,54.0,22.0,5.0,54.0,83.0,2.0,2.0,0.0,4.0,3.0,0,0,1986.0,23.0,1,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,14.3,0.771253,0.738239,0.733520,0.633492,1.255502,0.860715
11686,mccutan01,2010,262,1003.0,168.0,287.0,61.0,14.0,28.0,110.0,55.0,15.0,124.0,172.0,3.0,7.0,1.0,11.0,9.0,0,0,1986.0,24.0,2,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,13.3,0.875477,0.854333,0.855773,0.645223,1.569377,0.988228
11687,mccutan01,2011,420,1575.0,255.0,435.0,95.0,19.0,51.0,199.0,78.0,25.0,213.0,298.0,6.0,16.0,3.0,17.0,16.0,0,0,1986.0,25.0,3,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,12.3,0.885899,0.863263,1.039153,0.778178,1.483775,1.131680
11688,mccutan01,2012,577,2168.0,362.0,629.0,124.0,25.0,82.0,295.0,98.0,37.0,283.0,430.0,19.0,21.0,3.0,22.0,25.0,0,0,1986.0,26.0,4,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,11.3,0.943222,0.936194,1.253096,0.865186,1.398172,1.127696
11689,mccutan01,2013,734,2751.0,459.0,814.0,162.0,30.0,103.0,379.0,125.0,47.0,361.0,531.0,31.0,30.0,3.0,26.0,38.0,0,0,1986.0,27.0,5,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,10.3,0.956771,0.969236,1.259209,0.889235,1.426706,1.150807
11690,mccutan01,2014,880,3299.0,548.0,986.0,200.0,36.0,128.0,462.0,143.0,50.0,445.0,646.0,39.0,40.0,3.0,32.0,47.0,0,0,1986.0,28.0,6,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,9.3,0.951907,0.978365,1.304035,0.903312,1.360127,1.182154
11691,mccutan01,2015,1037,3865.0,639.0,1151.0,236.0,39.0,151.0,558.0,154.0,55.0,543.0,779.0,51.0,52.0,3.0,41.0,56.0,0,0,1986.0,29.0,7,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,8.3,0.951411,0.978932,1.318589,0.935155,1.255502,1.236423


In [88]:
active_hitters_ratios[active_hitters_ratios['playerID'] == 'troutmi01']

,playerID,yearID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,RBI_filled,SB_filled,birthYear,age,year,POS,DBE1,DBE2,SE,R_mean,H_mean,HR_mean,RBI_mean,SB_mean,BB_mean,retire_age_mean,yrs_remain,R_ratio,H_ratio,HR_ratio,RBI_ratio,SB_ratio,BB_ratio
17912,troutmi01,2011,40,123.0,20.0,27.0,6.0,0.0,5.0,16.0,4.0,0.0,9.0,30.0,0.0,2.0,0.0,1.0,2.0,0,0,1991.0,20.0,1,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,17.3,0.249319,0.192264,0.365561,0.224505,0.273032,0.171580
17913,troutmi01,2012,179,682.0,149.0,209.0,33.0,8.0,35.0,99.0,53.0,5.0,76.0,169.0,4.0,8.0,0.0,8.0,9.0,0,0,1991.0,21.0,2,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,16.3,0.928713,0.744134,1.279465,0.694564,1.808840,0.724450
17914,troutmi01,2013,336,1271.0,258.0,399.0,72.0,17.0,62.0,196.0,86.0,12.0,186.0,305.0,14.0,17.0,0.0,16.0,17.0,0,0,1991.0,22.0,3,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,15.3,1.072071,0.947080,1.510987,0.916731,1.956733,1.181998
17915,troutmi01,2014,493,1873.0,373.0,572.0,111.0,26.0,98.0,307.0,102.0,14.0,269.0,489.0,20.0,27.0,0.0,26.0,23.0,0,0,1991.0,23.0,4,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,14.3,1.162449,1.018289,1.791250,1.076925,1.740582,1.282087
17916,troutmi01,2015,652,2448.0,477.0,744.0,143.0,32.0,139.0,397.0,113.0,21.0,361.0,647.0,34.0,37.0,0.0,31.0,34.0,0,0,1991.0,24.0,5,OF,0,0,0,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,13.3,1.189251,1.059590,2.032521,1.114108,1.542633,1.376456


In [89]:
active_hitters_ratios[active_hitters_ratios['playerID'] == 'pujolal01'] 

,playerID,yearID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,RBI_filled,SB_filled,birthYear,age,year,POS,DBE1,DBE2,SE,R_mean,H_mean,HR_mean,RBI_mean,SB_mean,BB_mean,retire_age_mean,yrs_remain,R_ratio,H_ratio,HR_ratio,RBI_ratio,SB_ratio,BB_ratio
24255,pujolal01,2001,161,590.0,112.0,194.0,47.0,4.0,37.0,130.0,1.0,3.0,69.0,93.0,6.0,9.0,1.0,7.0,21.0,0,0,1980.0,21.0,1,1B,0,0,1,1357.6,2439.3,298.5,1453.2,147.7,948.6,37.4,16.4,1.435474,1.383840,2.156784,1.556565,0.117806,1.265655
24256,pujolal01,2002,318,1180.0,230.0,379.0,87.0,6.0,71.0,257.0,3.0,7.0,141.0,162.0,19.0,18.0,1.0,11.0,41.0,0,0,1980.0,22.0,2,1B,0,0,1,1357.6,2439.3,298.5,1453.2,147.7,948.6,37.4,15.4,1.473925,1.351740,2.069347,1.538604,0.176710,1.293169
24257,pujolal01,2003,475,1771.0,367.0,591.0,138.0,7.0,114.0,381.0,8.0,8.0,220.0,227.0,31.0,28.0,1.0,16.0,54.0,0,0,1980.0,23.0,3,1B,0,0,1,1357.6,2439.3,298.5,1453.2,147.7,948.6,37.4,14.4,1.567914,1.405239,2.215075,1.520644,0.314150,1.345140
24258,pujolal01,2004,629,2363.0,500.0,787.0,189.0,9.0,160.0,504.0,13.0,13.0,304.0,279.0,43.0,35.0,1.0,25.0,75.0,0,0,1980.0,24.0,4,1B,0,0,0,1357.6,2439.3,298.5,1453.2,147.7,948.6,37.4,13.4,1.602092,1.403456,2.331658,1.508671,0.382871,1.394054
24259,pujolal01,2005,790,2954.0,629.0,982.0,227.0,11.0,201.0,621.0,29.0,15.0,401.0,344.0,70.0,44.0,1.0,28.0,94.0,0,0,1980.0,25.0,5,1B,0,0,0,1357.6,2439.3,298.5,1453.2,147.7,948.6,37.4,12.4,1.612345,1.400959,2.343317,1.487118,0.683277,1.471094
24260,pujolal01,2006,933,3489.0,748.0,1159.0,260.0,12.0,250.0,758.0,36.0,17.0,493.0,394.0,98.0,48.0,1.0,31.0,114.0,0,0,1980.0,26.0,6,1B,0,0,0,1357.6,2439.3,298.5,1453.2,147.7,948.6,37.4,11.4,1.597820,1.377895,2.428811,1.512662,0.706838,1.507168
24261,pujolal01,2007,1091,4054.0,847.0,1344.0,298.0,13.0,282.0,861.0,38.0,23.0,592.0,452.0,120.0,55.0,1.0,39.0,141.0,0,0,1980.0,27.0,7,1B,0,0,0,1357.6,2439.3,298.5,1453.2,147.7,948.6,37.4,10.4,1.550825,1.369573,2.348313,1.472750,0.639520,1.551279
24262,pujolal01,2008,1239,4578.0,947.0,1531.0,342.0,13.0,319.0,977.0,45.0,26.0,696.0,506.0,154.0,60.0,1.0,47.0,157.0,0,0,1980.0,28.0,8,1B,0,0,0,1357.6,2439.3,298.5,1453.2,147.7,948.6,37.4,9.4,1.517181,1.365115,2.324372,1.462273,0.662661,1.595825
24263,pujolal01,2009,1399,5146.0,1071.0,1717.0,387.0,14.0,366.0,1112.0,61.0,30.0,811.0,570.0,198.0,69.0,1.0,55.0,180.0,0,0,1980.0,29.0,9,1B,0,0,0,1357.6,2439.3,298.5,1453.2,147.7,948.6,37.4,8.4,1.525192,1.360855,2.370519,1.479402,0.798465,1.652892
24264,pujolal01,2010,1558,5733.0,1186.0,1900.0,426.0,15.0,408.0,1230.0,75.0,34.0,914.0,646.0,236.0,73.0,1.0,61.0,203.0,0,0,1980.0,30.0,10,1B,0,0,0,1357.6,2439.3,298.5,1453.2,147.7,948.6,37.4,7.4,1.520065,1.355307,2.378291,1.472750,0.883548,1.676534


In [91]:
elig_hitters_ratios.head(5)

,playerID,yearID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,RBI_filled,SB_filled,birthYear,age,year,POS,DBE1,DBE2,SE,inducted,R_mean,H_mean,HR_mean,RBI_mean,SB_mean,BB_mean,retire_age_mean,yrs_remain,R_ratio,H_ratio,HR_ratio,RBI_ratio,SB_ratio,BB_ratio
0,aaronha01,1954,122,468.0,58.0,131.0,27.0,6.0,13.0,69.0,2.0,2.0,28.0,39.0,NaN,3.0,6.0,4.0,13.0,0,0,1934.0,20.0,1,OF,0,0,0,1,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,17.3,0.723025,0.932838,0.950459,0.968180,0.136516,0.533806
1,aaronha01,1955,275,1070.0,163.0,320.0,64.0,15.0,40.0,175.0,5.0,3.0,77.0,100.0,5.0,6.0,13.0,8.0,33.0,0,0,1934.0,21.0,2,OF,0,0,0,1,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,16.3,1.015974,1.139344,1.462245,1.227764,0.170645,0.733983
2,aaronha01,1956,428,1679.0,269.0,520.0,98.0,29.0,66.0,267.0,7.0,7.0,114.0,154.0,11.0,8.0,18.0,15.0,54.0,0,0,1934.0,22.0,3,OF,0,0,0,1,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,15.3,1.117779,1.234289,1.608470,1.248812,0.159269,0.724450
3,aaronha01,1957,579,2294.0,387.0,718.0,125.0,35.0,110.0,399.0,8.0,8.0,171.0,212.0,26.0,8.0,18.0,18.0,67.0,0,0,1934.0,23.0,4,OF,0,0,0,1,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,14.3,1.206080,1.278201,2.010587,1.399651,0.136516,0.815007
4,aaronha01,1958,732,2895.0,496.0,914.0,159.0,39.0,140.0,494.0,12.0,9.0,230.0,261.0,42.0,9.0,18.0,21.0,88.0,0,0,1934.0,24.0,5,OF,0,0,0,1,1468.0,2569.9,250.3,1304.2,268.1,959.9,37.3,13.3,1.236621,1.301700,2.047143,1.386321,0.163819,0.876966


In [94]:
set(elig_hitters_ratios.columns) - set(active_hitters_ratios.columns)

{'inducted'}

In [93]:
len(active_hitters_ratios.columns)

42

In [104]:
with open('active_hitters.pkl', 'w') as f:
    pickle.dump(active_hitters_ratios['playerID'], f)
